# LTSM Using TensorFlow and Keras

We discussed the theory of LTSM and Recurrant Neural Networks in previous videos. In this lab, we will solve a sequence classification problem by building a RNN with LTSM using Keras and TensorFlow.

For this example, we will use the IMDB dataset in Keras which contains a set of movie reviews with their associated sentiment. Sentiments could be either positive or negative. Thus, the problem we are solving is a binary classification problem.

The IMDB dataset consists of 25000 movie reviews for training and 25000 for testing. The problem is to predict whether a review has a positive or negative sentiment.

Before we build our model, we need to import the necessary libraries to perform our classification.

In [59]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

#import numpy and matplotlib
import numpy as np

Now that we imported TensorFlow library and Keras, we can start using them. Keras contains a version of the IMDB dataset which we will use in this lab. The next cell will read the IMDB dataset.

In [60]:
data = tf.keras.datasets.imdb

After loading the data, we will split it into trining and testing.

In [61]:
num_of_words=10000
(x_train, y_train), (x_test, y_test) = data.load_data(num_words=num_of_words)

We not check what is in the data

In [62]:
print(x_train[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


We notice that the input is numbers and not word! Well the words are encoded as a sequence of word indexes where words are indexed by overall frequency. For example, "3" encodes the 3rd most frequent word in the data.

In [63]:
# A dictionary mapping words to an integer index
word_index = tf.keras.datasets.imdb.get_word_index()

# The first indices are reserved
word_index = {k:(v+3) for k,v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2  # unknown
word_index["<UNUSED>"] = 3

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

We now decode the training data using the code above.

In [64]:
decode_review(x_train[0])

"<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for wh

As we learned in the lesson, we need to alter the input sequences so that they all have the same length for modeling. For this, we will use the preprocessing library within keras.

In [65]:
# Only consider the first 400 words within the review
max_review_length = 400
x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=max_review_length)
x_test = keras.preprocessing.sequence.pad_sequences(x_test, maxlen=max_review_length)

We are not ready to build our model. We will have sequential layers with an input layer, an LTSM layer, and dense output layer. The input layer is of size 32 for each input word. The second layer is an LTSM layer with size 100, and finally one output node since we have a binary classification problem.

In [66]:
# Construct our model
embedding_vecor_length = 32
model = keras.models.Sequential()
model.add(keras.layers.Embedding(num_of_words, embedding_vecor_length, input_length=max_review_length))
model.add(keras.layers.LSTM(100))
model.add(keras.layers.Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=3, batch_size=64)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 400, 32)           320000    
_________________________________________________________________
lstm_10 (LSTM)               (None, 100)               53200     
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 101       
Total params: 373,301
Trainable params: 373,301
Non-trainable params: 0
_________________________________________________________________
None


C:\Users\mam_0\Anaconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 25000 samples, validate on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 327s 13ms/step - loss: 0.4901 - acc: 0.7718 - val_loss: 0.3800 - val_acc: 0.8473
Epoch 2/3
25000/25000 [==============================] - 318s 13ms/step - loss: 0.2896 - acc: 0.8864 - val_loss: 0.3318 - val_acc: 0.8646
Epoch 3/3
25000/25000 [==============================] - 334s 13ms/step - loss: 0.2127 - acc: 0.9208 - val_loss: 0.3338 - val_acc: 0.8575


Now that we have trained our model, lets look at the testing accuracy.

In [67]:
# Evaluate model
scores = model.evaluate(x_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 85.75%


In this lab, we learned how to load data from keras datasets, pad and trim data to a certain size, build an LTSM model, and evaluation test set.